In [1]:
#This one for Memba compartivie Networks 

import torch
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from loader import *


from models.Unet.unet_model import UNet
#from models.transnorm.vit_seg_modeling import VisionTransformer as ViT_seg
#from models.transnorm.vit_seg_modeling import CONFIGS as CONFIGS_ViT_seg
#from models.AttUnet.AttUnet import R2U_Net, AttU_Net, R2AttU_Net
#from models.UViT.UViT import UViT


from engine import *
import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # "0, 1, 2, 3"

from utils import *
from configs.config_setting_others import setting_config

import warnings
warnings.filterwarnings("ignore")



def main(config):
    work_dir =  '/users/lalit47/gnet_seg/results/unet_Remote_Monday_27_January_2025_14h_08m_00s/'

    print('#----------Creating logger----------#')
    sys.path.append(work_dir + '/')
    log_dir = os.path.join(work_dir, 'log')
    #checkpoint_dir = os.path.join(work_dir, 'checkpoints')
    #resume_model = os.path.join(checkpoint_dir, 'latest.pth')
    csv_folder = os.path.join(config.work_dir, 'csv_folder')
    # if not os.path.exists(checkpoint_dir):
    #     os.makedirs(checkpoint_dir)
    if not os.path.exists(csv_folder):
        os.makedirs(csv_folder)

    global logger
    logger = get_logger('test', log_dir)

    log_config_info(config, logger)





    print('#----------GPU init----------#')
    set_seed(config.seed)
    gpu_ids = [0]# [0, 1, 2, 3]
    torch.cuda.empty_cache()





    print('#----------Preparing dataset----------#')
 
    test_dataset = RemSemLoader(root_dir = config.data_path, train = False, test = True)
    test_loader = DataLoader(test_dataset,
                                batch_size=1,
                                shuffle=False,
                                pin_memory=True, 
                                num_workers=config.num_workers,
                                drop_last=True)




    print('#----------Prepareing Models----------#')
    model_cfg = config.model_config
    


    
    
    model = UNet(n_classes=model_cfg['num_classes'], n_channels=model_cfg['input_channels'])
      
    #model = AttU_Net(output_ch=model_cfg['num_classes'], img_ch=model_cfg['input_channels'])

    ####---------  Vision Transformer  based  TransNorm model ------------####
    # vit_name = 'R50-ViT-B_16'
    # vit_patches_size = 16
    # img_size = config.input_size_h
    # config_vit = CONFIGS_ViT_seg['R50-ViT-B_16']
    # config_vit.n_classes = model_cfg['num_classes']
    # config_vit.n_skip = 3
    # if vit_name.find('R50') != -1:
    #     config_vit.patches.grid = (int(img_size / vit_patches_size), int(img_size / vit_patches_size))
    # model = ViT_seg(config_vit, img_size=img_size, num_classes=config_vit.n_classes).to('cuda')
    

           
    ##-UVIT-------------------------------------------##
    # in_bands = model_cfg['input_channels']  # Example value for in_bands
    # N = [3, 6, 9, 12]  # Example value for N
    # def get_uvit(in_channels,num_classes,in_bands, N,type,img_size=224):
    # return UViT(
    #     in_channels=in_channels,
    #     out_channels=num_classes,
    #     in_bands = in_bands,
    #     img_size=img_size,
    #     n=N,
    #     freeze_encoder=False,
    #     type = type,
    #     ckpt_path = "/models/Prithvi_100M.pt",
    #     cfg_path = "/models/config.yaml"
    # ).to('cpu')
    
    # model = get_uvit(in_channels = model_cfg['input_channels'], num_classes = = model_cfg['num_classes']), in_bands, N, type, img_size = config.input_size_h)
    
    model = torch.nn.DataParallel(model.cuda(), device_ids=gpu_ids, output_device=gpu_ids[0])

   
    
    






    print('#----------Prepareing loss, opt, sch and amp----------#')
    criterion = config.criterion
    optimizer = get_optimizer(config, model)
    scheduler = get_scheduler(config, optimizer)
    scaler = GradScaler()





    print('#----------Set other params----------#')
    min_loss = 999
    start_epoch = 1
    min_epoch = 1




   #=-----------------To Resume the model---------------#
    # if os.path.exists(resume_model):
    #     print('#----------Resume Model and Other params----------#')
    #     checkpoint = torch.load(resume_model, map_location=torch.device('cpu'))
    #     model.module.load_state_dict(checkpoint['model_state_dict'])
    #     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    #     scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    #     saved_epoch = checkpoint['epoch']
    #     start_epoch += saved_epoch
    #     min_loss, min_epoch, loss = checkpoint['min_loss'], checkpoint['min_epoch'], checkpoint['loss']

    #     log_info = f'resuming model from {resume_model}. resume_epoch: {saved_epoch}, min_loss: {min_loss:.4f}, min_epoch: {min_epoch}, loss: {loss:.4f}'
    #     logger.info(log_info)





    
    print('#----------Testing----------#')
    best_weight = torch.load(work_dir + 'checkpoints/best.pth', map_location=torch.device('cpu'))
    model.module.load_state_dict(best_weight)
    loss = test_all_images(
            test_loader,
            model,
            criterion,
            logger,
            config,
            csv_folder,
        )
           


if __name__ == '__main__':
    config = setting_config
    main(config)

#----------Creating logger----------#
#----------GPU init----------#
#----------Preparing dataset----------#
#----------Prepareing Models----------#
#----------Prepareing loss, opt, sch and amp----------#
#----------Set other params----------#
#----------Testing----------#


100%|██████████| 1/1 [00:00<00:00,  2.83it/s]

Test results saved to results/unet_Remote_Wednesday_19_February_2025_17h_05m_12s/csv_folder/test_results.csv
